In [1]:
import pydub # faster than librosa
import numpy      as np
import pandas     as pd
import time

from pytorch.coughvid_dataset import CoughvidDataset
from pytorch.test_coughvid_dataset import TestCoughvidDataset
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.models import resnet50, resnet18

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy


#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter()

In [2]:
dir = 'C:/COUGHVID_public_dataset/public_dataset/'

full_dataset = CoughvidDataset(dir, 'metadata_compiled.csv', get_features=True)

dataframe = full_dataset.dataframe

minority_class_count = len(dataframe[dataframe['status']==2])

print(f'{minority_class_count} samples in the minority class.')

Mask file C:/COUGHVID_public_dataset/public_dataset/ does not exist. Calculating masks on the fly.
8997 records ready to load across 2 groups.
699 samples in the minority class.


In [3]:
sample_dataset = CoughvidDataset(dir, 'metadata_compiled.csv', get_features=True, samples_per_class=400)


Mask file C:/COUGHVID_public_dataset/public_dataset/ does not exist. Calculating masks on the fly.
800 records ready to load across 2 groups.


In [4]:
# split data into training and test samples
train_indices, test_indices = train_test_split(np.arange(0,len(sample_dataset)-1), test_size=0.25)

batch_size  = 1
num_workers = 2

train_loader  = DataLoader(sample_dataset, 
                           num_workers=num_workers,
                           sampler=SubsetRandomSampler(train_indices)
                          )

test_loader   = DataLoader(sample_dataset, 
                           num_workers=num_workers,
                           sampler=SubsetRandomSampler(test_indices)
                          )

dataloaders = {
    "train": train_loader,
    "test": test_loader
}

In [9]:
# load model and change output shape for binary prediction
model = resnet18()

model.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=512, #2048 for resnet50
        out_features=1
    ),
    torch.nn.Sigmoid()
)

model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=True)

model.double()

optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.BCELoss()

In [7]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
batch_size = 100
n_iters = 3000
EPOCHS = int(n_iters / (len(sample_dataset) / batch_size))

lr_rate = 0.00001    
    
model = LogisticRegression(4050,1)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)
criterion = torch.nn.BCELoss()
model.double()


LogisticRegression(
  (linear): Linear(in_features=4050, out_features=1, bias=True)
)

In [10]:
'''RESNET training code adapted from https://www.kaggle.com/gxkok21/resnet50-with-pytorch'''

#if torch.cuda.is_available():
#    model = model.cuda()
#    print('Using GPU.')

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

EPOCHS = 10

for i in range(EPOCHS):
    for phase in ["train", "test"]:
        if phase == "train":
            model.train()
        else:
            model.eval()
        
        samples = 0
        loss_sum = 0
        correct_sum = 0
        for j, batch in enumerate(dataloaders[phase]):
            X,labels = batch
            #print(labels)
            #print(len(X),len(X[0]),len(X[1]))
            #print(labels[None,...].double())
            #X = torch.Tensor(X)
            #print(X.shape)
            #labels = 1-labels
            if torch.cuda.is_available():
                X = X.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                y = model(X[None,...].double())
                #y = model(X.flatten().double())
                print(y,labels)
                loss = criterion(
                    y,
                    #torch.clip(y,0,1), 
                    labels[None,...].double()
                    #labels.double()
                )
                #print(loss.item())
                #print(labels[None,...].double())
                
                #writer.add_scalar("Loss/train", loss, i)

                if phase == "train":
                    loss.backward()
                    optimizer.step()
                    
                loss_sum += loss.item() * X.shape[0] # We need to multiple by batch size as loss is the mean loss of the samples in the batch
                samples += X.shape[0]
                num_corrects = torch.sum((y >= 0.5).float() == labels[0].float())
                correct_sum += num_corrects
                
                # Print batch statistics every 50 batches
                if j % 50 == 49 and phase == "train":
                    print("{}:{} - loss: {}, acc: {}".format(
                        i + 1, 
                        j + 1, 
                        float(loss_sum) / float(samples), 
                        float(correct_sum) / float(samples)
                    ))
                
        # Print epoch statistics
        epoch_acc = float(correct_sum) / float(samples)
        epoch_loss = float(loss_sum) / float(samples)
        print("epoch: {} - {} loss: {}, {} acc: {}".format(i + 1, phase, epoch_loss, phase, epoch_acc))
        
        # Deep copy the model
        if phase == "test" and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "resnet50.pth")

tensor([[0.7916]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.9275]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.9908]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.8096]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.8780]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.9287]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.7076]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.3364]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.1472]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.1972]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0

tensor([[0.4139]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.3541]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.3270]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.2932]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.2561]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.2186]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.1834]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.1743]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.1857]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.1918]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0

tensor([[0.6001]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.5896]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.5932]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.6094]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.6046]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.6128]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.6008]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.6026]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.5850]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.5824]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0

tensor([[0.6684]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.6972]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
1:250 - loss: 0.8087710619696075, acc: 0.46
tensor([[0.7030]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.7162]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.7089]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.7101]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.7188]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.7076]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.6777]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.6296]], dtype=torch.float64, grad_fn=<SigmoidBackward0

tensor([[0.6104]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.5675]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.5410]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.5307]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.5056]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.4978]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.5058]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.5277]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.5317]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.5194]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0

tensor([[0.8326]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.8341]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.8381]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.8279]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.8214]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.8003]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.7635]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.7322]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.7085]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.6682]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0

tensor([[0.3976]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.4061]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.4299]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.4673]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.5160]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
1:500 - loss: 0.7767813416329024, acc: 0.47
tensor([[0.5454]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.5838]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.6021]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([1], dtype=torch.int32)
tensor([[0.6287]], dtype=torch.float64, grad_fn=<SigmoidBackward0>) tensor([0], dtype=torch.int32)
tensor([[0.6358]], dtype=torch.float64, grad_fn=<SigmoidBackward0

KeyboardInterrupt: 